In [55]:
import pandas as pd
import os

In [15]:
#os.chdir('C://Users//adam//Desktop//dagger')

In [56]:
#import, clean and format HI data
hdi_file = 'data/Human Development Index (HDI).csv'
hdi_df = pd.read_csv(hdi_file, encoding = "ISO-8859-1", skiprows=1)
columns = ["Country"]

for i in range(1990, 2018):
   columns = columns + [str(i)]

hdi_df = hdi_df[columns]
hdi_df = hdi_df.melt(id_vars='Country', var_name='Year', value_name="HDI")
hdi_df = hdi_df.dropna()
hdi_df['Country'] = hdi_df['Country'].str.strip()
#hdi_df.head()

In [57]:
#import and clean the power data
power_file = 'data/global_power_plant_database.csv'
power_df = pd.read_csv(power_file)
power_df['commissioning_year'] = round(power_df['commissioning_year'],0)
power_df = power_df[power_df['commissioning_year']>=1990]
#power_df.head()

In [58]:
#importing, cleaning, and formatting the population data
pop_file = ('data/API_SP.POP.TOTL_DS2_en_csv_v2_103676.csv')
pop_df = pd.read_csv(pop_file, encoding = "ISO-8859-1", skiprows=4)
pop_df = pop_df.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1)
pop_df = pop_df.melt(id_vars='Country Name', var_name='Year', value_name='Population')
pop_df = pop_df.rename(columns={'Country Name':'Country'})
#pop_df.head()

In [59]:
#filter power df and create df for cumulative capacity
pow_pd = power_df[['country_long', 'capacity_mw', 'commissioning_year' ]]
pow_pd.head(20)
country_list = pow_pd['country_long'].unique()

cap_data = pd.DataFrame(columns = ['country_long', 'commissioning_year'])

for i in range(len(country_list)):
    for year in range(1990, 2018):
        cap_data.loc[i*37 + year-1990] = [country_list[i], year]    

# Aggregate capacity when same country and year
countries = pow_pd.groupby(['country_long', 'commissioning_year'])
cap_added = countries['capacity_mw'].sum()


In [67]:
# merge data agreggate capacity with counrty year df
cap_cont_yr = pd.merge(cap_data, cap_added, on=['country_long', 'commissioning_year'], how='left')
cap_cont_yr = cap_cont_yr.fillna(0)

In [61]:
# create cumulative cap by country and year
cap_cont_yr['commissioning_year'] = cap_cont_yr['commissioning_year'].astype(str)
cap_cum = cap_cont_yr.groupby(by=['country_long', 'commissioning_year']).sum().groupby(level=[0]).cumsum()

In [62]:
# merge hdi, pop, and capacity by Country and Year
hdi_pop_merged = pd.merge(hdi_df, pop_df, on=['Country', 'Year'])
merged_data = pd.merge(hdi_pop_merged, cap_cum, left_on=['Country', 'Year'], right_on=['country_long', 'commissioning_year'])

In [66]:
# reformat capacity added
cap_added_yr = cap_cont_yr.rename(columns ={'country_long':'Country', 'commissioning_year':'Year', 'capacity_mw':'cap_added' })


In [65]:
merged_data = pd.merge(merged_data, cap_added_yr, on=['Country', 'Year'])
merged_data[merged_data['Country'] == 'China']

,Country,Year,HDI,Population,capacity_mw,cap_added
11,China,1990,0.502,1.135185e+09,3530.0,3530.0
79,China,1991,0.510,1.150780e+09,6055.0,2525.0
147,China,1992,0.521,1.164970e+09,17635.0,11580.0
214,China,1993,0.531,1.178440e+09,21776.0,4141.0
282,China,1994,0.541,1.191835e+09,32226.0,10450.0
351,China,1995,0.550,1.204855e+09,43440.5,11214.5
420,China,1996,0.559,1.217550e+09,55600.5,12160.0
489,China,1997,0.568,1.230075e+09,66762.5,11162.0
558,China,1998,0.576,1.241935e+09,91219.5,24457.0
627,China,1999,0.585,1.252735e+09,104109.5,12890.0
